# Домашнее задание 3

В этом домашнем задании мы рассмотрим практическое применение модели SARIMA. 

Эта модель - самый универсальный базовый инструмент. Если вы не знаете, с чего начать, то это будет хорошим бейзлайном. Если у вас при этом ещё и мало данных, то скорее всего она будет чуть ли не единственным адекватным вариантом наряду с ETS. 


# 1. SARIMA как независимая модель

Для начала попробуем применить модель как самостоятельную на очень простых данных. Возьмём стандартный датасет квартальных данных по макроэкономике США и выберем из него ряд численности населения.

In [1]:
from sktime.datasets import load_macroeconomic
import pandas as pd 

data = load_macroeconomic()
data.index = pd.date_range(start = '01.01.1959', periods = len(data), freq='Q')
data = data['pop']

Исходя из описания на сайте statsmodels, он представляет собой численность населения США на конец каждого квартала с учётом всех вооружённых сил, находящихся за рубежом.

Для начала вспомним структуру модели. Она представлена на картинке. Нотация B (Backshift) эквивалентна оператору лага. Период сезонности равен четырём.

![Alt text](sarima_scheme.png)

Разберём ещё раз подробно параметры и как их подбирать.
$$SARIMA(p,d,q)(P,D,Q)$$

Начнём с базовых.

- p - порядок авторегрессии
- d - порядок интеграции
- q - порядок скользящего среднего

Когда мы говорили про ARIMA-модель на семинарах, мы обсуждали, что для каждого из них есть свой паттерн поведения автокорреляций:

- Для выбора параметра d мы используем тесты на стационарность. Также нестационарность можно видеть по характерному рисунку автокорреляций: ACF очень (иногда и не очень) медленно убывает, а из PACF очень значима первая, а остальные близки к нулю.
- Для AR(p)-процесса частные автокорреляции обрываются на лаге p
- Для MA(q)-процесса автокорреляции обрываются на лаге q
- Для ARMA(p,q) обе убывают и обычно на практике просто перебирают все комбинации, ограничивая границы перебора максимально значимым порядком автокорреляций.

Сезонные параметры подбираются очень похожим способом. Так как SARIMA - модель с одним периодом сезонности, зафиксируем его. Предположим, что у нас месячные данные и, соответственно, месячная сезонность.

- Если в нашем процессе сезонность представлена авторегрессией порядка P, то автокорреляции сезонных лагов (т.е. каждого 12-го лага) экспоненциально убывают, а частные автокорреляции сезонных лагов обрываются на порядке P.

- Если в нашем процессе сезонность представлена скользящим средним порядка Q, то автокорреляции сезонных лагов (т.е. каждого 12-го лага) обрываются на порядке Q, а частные автокорреляции сезонных лагов экспоненциально убывают.

То есть на самом деле, подход вообще не поменялся, и мы просто смотрим не на каждый лаг, а на каждый k-й лаг

Про параметр D пока что осознанно умолчим, о нём чуть ниже.

### 1.0 (0 баллов)

Разделите ряд на тренировочную и тестовую выборку, в качестве теста возьмите 20% последних наблюдений. Весь анализ производим на тренировочной выборке.

In [3]:
# ༼ つ ◕_◕ ༽つ

### 1.1 (0.5 балла)
Визуализируйте этот ряд, его автокорреляции и частные автокорреляции. Присутствуют ли в нём тренд и сезонность? 

In [ ]:
# ༼ つ ◕_◕ ༽つ

### 1.2 (0.5 балла)

Выясните порядок интеграции ряда (aka параметр d) c помощью KPSS-теста. Нарисуйте коррелограммы стационарной версии ряда.

In [ ]:
# ༼ つ ◕_◕ ༽つ

### 1.3 (0.5 балла)

По графикам автокорреляций из предыдущего пунтка подберите значения (или по крайней мере границы значений) параметров p, q, P, Q. Обязательно аргументируйте выбор каждого параметра. Без аргументации пункт не засчитывается.

In [ ]:
# ༼ つ ◕_◕ ༽つ

### Длинное лирическое отступление

На самом деле для рядов с сильной сезонностью можно производить отбор несколько по-другому и сэкономить пару параметров. Для этого начнём подбор не с параметра d, а с параметра D. 

Возникает первый вопрос. А что такое вообще "сильная" сезонность? Мы помним, что из STL-разложения можно подсчитать силу сезонности, но с каким порогом её сравнивать? В основном, пользуются эмпирической константой 0.65. Если сила сезонности больше неё, то сезонность считается сильной. В этом случае обычно пытаются однократно применить сезонные разности, что соответствует D=1. 

Сезонная разность порядка 12, например, выражается следующим образом:
$$ \Delta y^{12}_t = y_t - y_{t-12}$$

Притом важно обратить внимание, что сезонная разность выполняет двойную функцию.

Во-первых, она стандартным уменьшает порядок интеграции на единицу.

Во-вторых, она фильтрует значительную часть сезонных автокорреляций. Здесь логика абсолютно такая же, как с обычными автокорреляциями и трендами. Если привести ряд к стационарному, картина автокорреляций становится более очевидной. Вот и с сезонными лагами ситукция такая же. То есть взятие сезонных разностей может сократить количество необходимых сезонных коэффициентов

Так как сила сезонности после взятия разности уже почти наверное не будет высокой, на практике D=2 никогда не возникает. Если же после взятия сезонных разностей ряд всё ещё остаётся нестационарным, то можно дополнительно взять несколько раз обычные разности по стандартной процедуре. 

Теперь вернёмся к исходным данным и попробуем повторить подбор параметров.

### 1.4 (0.5 балла)

Подберите параметр D, а затем параметр d. Нарисуйте коррелограммы стационарной версии ряда.

In [4]:
# ༼ つ ◕_◕ ༽つ

### 1.5 (0.5 балла)

По графикам автокорреляций из предыдущего пунтка подберите значения (или по крайней мере границы значений) параметров p, q, P, Q. Стал ли выбор параметров проще и очевиднее? Обязательно аргументируйте выбор каждого параметра. Без аргументации пункт не засчитывается.

In [5]:
# ༼ つ ◕_◕ ༽つ

### 1.6 (0.5 балла)

Сравните на кросс-валидации две полученных модели и модель, выбранную автоматическим алгоритмом. Скорее всего результаты моделей будут примерно эквивалентны. Горизонт прогнозирования: 12 месяцев.

In [6]:
# ༼ つ ◕_◕ ༽つ

### 1.7 (0.5 балла)

Постройте прогнозы трёх моделей на тестовый период и визуализируйте их на графике вместе с тестовыми данными. Желательно ещё на графике захватывать несколько лет трейна. Почему-то в прошлых дз почти никто так не делал, но это довольно неплохо помогает визуально оценивать адекватность прогнозов.

In [ ]:
# ༼ つ ◕_◕ ༽つ

# SARIMA как вспомогательная модель

В предыдущем ДЗ мы рассматривали модели ETS и STL как базовые, а на их остатки накладывали модели с признаками. На самом деле, можно поступить наоборот. Для начала построить какую-нибудь хорошую базовую модель с признаками (например, многошаговый бустинг), а потом подбить результат, навесив на остатки SARIMA. Давайте обсудим, в каком случае такой подход подойдёт.

Дисклеймер: Дальше идёт субъективный личный опыт преподавателя, если у вас есть свои соображения на эту тему, пишите в чат

Вообще, мой подход к рядам базируется на следующем: базовую модель следует выбирать по основным характеристикам ряда. Приведу примеры.

- Пусть некоторый временной ряд имеет сильные структурные паттерны (тренд, сезонность и т.п.). В таком случае лучше взять за основу структурные модели вроде ETS, SARIMA и так далее. Если качество моделей не устроит, они всё равно неплохо отфильтруют базовые паттерны, а моделью на остатках мы добьём то, что осталось. Собственно, так мы делали в предыдущей ДЗ.

- Пусть некоторый временной ряд не имеет сильных паттернов, но ощутимо коррелирует с некоторыми другими рядами. Тогда лучше сначала построить признаковую модель, а на её остатках уже обучить какой-то базовый алгоритм. Если вы хорошо подобрали признаки, то остатки скорее всего будут стационарными, а в этом случае прекрасно подойдёт SARIMA. Это мы и попробуем сейчас.

Теперь поговорим про данные. В этот раз они недельные. Будем прогнозировать на небольшой горизонт, 8 недель.

* target -- целевая переменная, средневзвешенная по портам стоимость аммиака (usd/tonn) в некотором европейском регионе. Аммиак -- один из основных компонентов для производства азотных удобрений.

* oil -- стоимость фьючерса на нефть марки BRENT (usd)
* gas -- стоимость фьючерса на природный газ (usd)
* coal -- стоимость фьючерса на уголь (usd)
* corn, soybean, wheat, rice -- стоимость фьючерса на  кукурузу, сою, пшеницу и рис соответственно (usd)

* usd_eur, usd_rub, usd_jpy, usd_cny -- курсы евро, рубля, йены и юаня к доллару
* usd_index -- индекс доллара

* cpy_us, cpi_china -- ВВП США и Китая

Также для простоты будем использовать для признаковой модели прямую стратегию, а для SARIMA -- рекурсивную. План действий будет примерно следующий.

1. Строим признаковую модель. Возьмите любую, которая вам больше нравится. Так как мы используем прямую стратегию, придётся построить 8 моделей (по одной на каждый горизонт).
2. Прогнозируем на 8 шагов вперёд с помощью стратегии
3. Берём из стратегии модель первого шага и с помощью неё достаём остатки на трейне.
4. На этих остатках оцениваем SARIMA-модель и рекурсивно прогнозируем на 8 шагов вперёд.
5. Складываем прогнозы шагов 2 и 4, и сравниваем их с тестовой выборкой
6. Дополнительно проводим кросс-валидацию на трейне. Бейзлайн -- чистая SARIMA.

### 2.0 (0 баллов)

Разбейте выборку на трейн и тест. В качестве теста возьмите последние 8 наблюдений.

In [ ]:
# ༼ つ ◕_◕ ༽つ

### 2.1 (3 балла)

Постройте признаковую модель с прямой стратегией. В выборе самой модели вы не ограничены. За использование дополнительных переменных может быть добавлен бонус на усмотрение ассистента. Постройте прогноз на 8 шагов вперёд.

In [ ]:
# ༼ つ ◕_◕ ༽つ

### 2.2 (1.5 балла)

Достаньте остатки из одношаговой модели предыдущего пункта. В идеале они должны остаться стационарными. Если остатки представляют собой белый шум, уберите из регрессоров в предыдущем пункте все лаги таргета. Предположите параметры SARIMA-модели, обоснуйте свои предположения и постройте рекурсивный прогноз на 8 шагов вперёд

In [ ]:
# ༼ つ ◕_◕ ༽つ

### 2.3 (1 балл)

Постройте суммарный прогноз двух моделей, а также бейзлайн-прогноз чистой автоматической SARIMA. Визуализируйте оба прогноза и сравните их точность

In [ ]:
# ༼ つ ◕_◕ ༽つ

### 2.4 (1 балл)
Сравните точность модели с бейзлайном на кросс-валидации. Возьмите любой способ кросс-валидации, который сочтёте подходящим. Горизонт прогнозирования: 8 шагов

In [ ]:
# ༼ つ ◕_◕ ༽つ

##### Рубрика "как вам домашка?" (0.1 балла)

Пройдите короткий опрос. Это действительно важно. https://forms.gle/w3sV453spERTbGvr7